# Estimación de dirección de arribo de fuentes sonoras 

In [1]:
import numpy as np
import pandas as pd
import gspread # Extension para leer el excel desde Drive
from oauth2client.service_account import ServiceAccountCredentials # Extensión para autentificación con Drive
from tdoa_py import estimation, simulation_room

In [2]:
# Configurar credenciales
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name("simulationsdoa-credenciales.json", scope)
client = gspread.authorize(creds)

# Abrir la hoja de cálculo
spreadsheet = client.open_by_url("https://docs.google.com/spreadsheets/d/13XTDng98P99pfexK78Dd4Gud1CzZwO7PfVhpyIG1jCM/edit?gid=0#gid=0")
sheet = spreadsheet.sheet1

# Leer datos
data = sheet.get_all_records()
df_simulations = pd.DataFrame(data)

df_simulations["room_dim"] = df_simulations.apply(lambda row: (row["room_x"], row["room_y"], row["room_z"]), axis=1) # Para pasar el recinto en una tupla luego
df_simulations.head() # Verificar el resultado

,sim_name,method,audio,room_x,room_y,room_z,rt60,snr_db,n_mics,mic_d,...,src_az_deg,src_z,theta_1,theta_2,theta_3,avg_angle_deg,avg_tdoa,hemi_1,hemi_2,room_dim
0,1,classic,audio_anecoico_corto.wav,100,100,100,3,70,4,0.1,...,0,1.2,,,,0.0,0.0,,,"(100, 100, 100)"
1,1,classic,audio_anecoico_corto.wav,100,100,100,3,70,4,0.1,...,5,1.2,,,,0.0,0.0,,,"(100, 100, 100)"
2,1,classic,audio_anecoico_corto.wav,100,100,100,3,70,4,0.1,...,10,1.2,,,,4.1,0.0,,,"(100, 100, 100)"
3,1,classic,audio_anecoico_corto.wav,100,100,100,3,70,4,0.1,...,15,1.2,,,,16.5,0.0,,,"(100, 100, 100)"
4,1,classic,audio_anecoico_corto.wav,100,100,100,3,70,4,0.1,...,20,1.2,,,,21.7,0.0,,,"(100, 100, 100)"


In [3]:
# Iterar sobre el DataFrame con indexación correcta
for idx, sim in df_simulations.iterrows():
    src_az_deg, mic_signals = simulation_room.sim_room_Nmics(
        wav_path=f'audios/anechoic/{sim["audio"]}',
        out_dir=f'audios/output/{sim["sim_name"]}',
        fs=48000,
        room_dim=sim["room_dim"],
        rt60=sim["rt60"],
        snr_db=sim["snr_db"],
        n_mics=sim["n_mics"],
        mic_d=sim["mic_d"],
        mic_z=sim["mic_z"],
        mic_directivity=sim["mic_directivity"],
        src_dist=sim["src_dist"],
        src_az_deg=sim["src_az_deg"],
        src_z=sim["src_z"],
        save_audio=False
    )
    
    phi_deg, tdoas = estimation.estimate_doa(
        signals=mic_signals,
        fs=48000,
        d=sim["mic_d"],
    )

    # Guardar resultados en cada fila individual
    
    df_simulations.at[idx, 'avg_angle_deg'] = phi_deg
    df_simulations.at[idx, 'avg_tdoa'] = tdoas[0]
    # df_simulations.at[idx, 'hemi_1'] = hemi_avgs['H1']
    # df_simulations.at[idx, 'hemi_2'] = hemi_avgs['H2']
    # df_simulations.at[idx, 'theta_1'] = angles[0]
    # df_simulations.at[idx, 'theta_2'] = angles[1]
    # df_simulations.at[idx, 'theta_3'] = angles[2]
    print("phi_deg", phi_deg)
    print("tdoas", tdoas)

# # Convertir el DataFrame a una lista de listas para Google Sheets
df_simulations["room_dim"] = df_simulations["room_dim"].apply(lambda x: ",".join(map(str, x)))
data = [df_simulations.columns.values.tolist()] + df_simulations.values.tolist()

# Escribir en la hoja de cálculo
sheet.update(data)
print("DataFrame guardado correctamente en Google Sheets.")
df_simulations



posicion micrófonos:  [[49.85 49.95 50.05 50.15]
 [50.   50.   50.   50.  ]
 [ 1.2   1.2   1.2   1.2 ]]
posicion fuente:  [np.float64(70.0), np.float64(50.0), 1.2]
phi_deg 0.0
tdoas [np.float64(-0.0002916666666666667), np.float64(-0.0005833333333333334), np.float64(-0.000875)]
posicion micrófonos:  [[49.85 49.95 50.05 50.15]
 [50.   50.   50.   50.  ]
 [ 1.2   1.2   1.2   1.2 ]]
posicion fuente:  [np.float64(69.9238939618349), np.float64(51.74311485495316), 1.2]
phi_deg 0.0
tdoas [np.float64(-0.0002916666666666667), np.float64(-0.0005833333333333334), np.float64(-0.000875)]
posicion micrófonos:  [[49.85 49.95 50.05 50.15]
 [50.   50.   50.   50.  ]
 [ 1.2   1.2   1.2   1.2 ]]
posicion fuente:  [np.float64(69.69615506024417), np.float64(53.47296355333861), 1.2]
phi_deg 4.140003537900071
tdoas [np.float64(-0.0002916666666666667), np.float64(-0.0005833333333333334), np.float64(-0.0008541666666666667)]
posicion micrófonos:  [[49.85 49.95 50.05 50.15]
 [50.   50.   50.   50.  ]
 [ 1.2   1.2

,sim_name,method,audio,room_x,room_y,room_z,rt60,snr_db,n_mics,mic_d,...,src_az_deg,src_z,theta_1,theta_2,theta_3,avg_angle_deg,avg_tdoa,hemi_1,hemi_2,room_dim
0,1,classic,audio_anecoico_corto.wav,100,100,100,3,70,4,0.1,...,0,1.2,,,,0.000000,-0.000292,,,"100,100,100"
1,1,classic,audio_anecoico_corto.wav,100,100,100,3,70,4,0.1,...,5,1.2,,,,0.000000,-0.000292,,,"100,100,100"
2,1,classic,audio_anecoico_corto.wav,100,100,100,3,70,4,0.1,...,10,1.2,,,,4.140004,-0.000292,,,"100,100,100"
3,1,classic,audio_anecoico_corto.wav,100,100,100,3,70,4,0.1,...,15,1.2,,,,16.472908,-0.000271,,,"100,100,100"
4,1,classic,audio_anecoico_corto.wav,100,100,100,3,70,4,0.1,...,20,1.2,,,,21.726984,-0.000271,,,"100,100,100"
5,1,classic,audio_anecoico_corto.wav,100,100,100,3,70,4,0.1,...,25,1.2,,,,24.534373,-0.000271,,,"100,100,100"
6,1,classic,audio_anecoico_corto.wav,100,100,100,3,70,4,0.1,...,30,1.2,,,,30.962969,-0.000250,,,"100,100,100"
7,1,classic,audio_anecoico_corto.wav,100,100,100,3,70,4,0.1,...,35,1.2,,,,36.279353,-0.000229,,,"100,100,100"
8,1,classic,audio_anecoico_corto.wav,100,100,100,3,70,4,0.1,...,40,1.2,,,,39.968308,-0.000229,,,"100,100,100"
9,1,classic,audio_anecoico_corto.wav,100,100,100,3,70,4,0.1,...,45,1.2,,,,44.390938,-0.000208,,,"100,100,100"
